## Lab Seven: Sequential Networks
GROUP MEMBERS:
- **Alex Chen** 
- **Paige Maple** 
- **Sam Valentine**

### Sources
1. ChatGPT (For formatting text and plots)

### Preparation (3 pts total)

We choose the ... dataset ...

#### Part One (1 pts)

#### Part Two (1 pts)

#### Part Three (1 pts)

### Modeling (6 pts total)

#### Part One (3 pts)

#### Part Two (1 pts)

#### Part Three (2 pts)

### Exceptional Work (1 pts)
Use the pre-trained ConceptNet Numberbatch embedding and compare to the embedding you used above (it is fine to use either a CNN or a transformer for this section). Which method is better for your specific application? 